# ExploreAI Academy Classification Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**Ntshagovhe Rofhiwa**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

## Predict Overview: EA- language classification
South Africa is a multicultural society that is characterised by its rich linguistic diversity. Language is an indispensable tool that can be used to deepen democracy and also contribute to the social, cultural, intellectual, economic and political life of the South African society.

The country is multilingual with 11 official languages, each of which is guaranteed equal status. Most South Africans are multilingual and able to speak at least two or more of the official languages. From South African Government. With such a multilingual population, it is only obvious that our systems and devices also communicate in multi-languages.

In this challenge, you will take text which is in any of South Africa's 11 Official languages and identify which language the text is in. This is an example of NLP's Language Identification, the task of determining the natural language that a piece of text is written in.


# Data Overview
The dataset used for this challenge is the NCHLT Text Corpora collected by the South African Department of Arts and Culture & Centre for Text Technology (CTexT, North-West University, South Africa). The training set was improved through additional cleaning done by Praekelt.

## Data Source
The data for this dataset was obtained from the EA- language classification challenge, provided by Kaggle(https://www.kaggle.com/competitions/south-african-language-identification-hack-2023/data).


## Table of Contents

1. Importing Packages

2. Loading Data

3. Data Engineering

4. Exploratory Data Analysis (EDA)

5. Modeling

6. Model Evaluation

7. References

# 1. Importing Packages

First we import the packages for cleaning,visualisation,analysis and modelling

In [247]:
import pandas as pd
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
special = string.punctuation 
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from scipy import stats 
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline

#Importing other libraries
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
import gensim
from collections import defaultdict
from collections import  Counter
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
additional = ['is']
stop = set().union(stopwords.words('english'),additional)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ntsha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ntsha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 2. Loading Data

In  this section the training and testing data will be loaded 

In [217]:
df = pd.read_csv("train_set.csv")


In [218]:
df_test = pd.read_csv("test_set.csv")

In [219]:
df_test

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
...,...,...
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."


## 3. Data Engineering

We now look at the train and test data to see what they contain.

In [220]:
df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [221]:
df_test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


### Checking for missing values

In [223]:
np.sum(df.isnull().any(axis=1))

0

### Checking for Duplicates

In [224]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
32995    False
32996     True
32997    False
32998    False
32999    False
Length: 33000, dtype: bool

### Data Types

In [225]:
df.dtypes

lang_id    object
text       object
dtype: object

## 4. Exploratory Data Analysis (EDA)

In [254]:
df.describe()  # provides statistical summary of the numerical columns in the DataFrame

,lang_id,text,clean_text
count,33000,33000,33000
unique,11,29937,29931
top,xho,ngokwesekhtjheni yomthetho ophathelene nalokhu unelungelo lokudlulisela isililo sakho kusomkhandlu wezehlalakuhle ngokutlola incwadi uyithumele eadresini elandelako kungakapheli amalanga amatjhumi...,ngokwesekhtjheni yomthetho ophathelene nalokhu unelungelo lokudlulisela isililo sakho kusomkhandlu wezehlalakuhle ngokutlola incwadi uyithumele eadresini elandelako kungakapheli amalanga amatjhumi...
freq,3000,17,17


### Data Cleaning 

In [248]:
#tokenize
def rem2(text) : 
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = text.lower()
    text = ' '.join(tknzr.tokenize(text))
    return (text)


# remove URLs
import re
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# remove htmls
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


# remove punct
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

# remove other characters
def remove_other (text) : 
    text = text.lower()
    text = text.replace("'", "")
    text = text.replace("  ", " ")
    text = text.replace("   ", " ")
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return(text)

# function to stemm the text
def stemming (text) : 
    stemmer = nltk.stem.PorterStemmer()
    return(" ".join(stemmer.stem(word) for word in text.split()))

# function to lemm a text 
def lemming (text) : 
    lemmatizer=nltk.stem.WordNetLemmatizer()
    return(" ".join(lemmatizer.lemmatize(word) for word in text.split()))

# Full clean ... combination off all the previous functions
def Full_clean (text) :
    text = remove_html(text)
    text = remove_URL(text)
    text = remove_punct(text)
    text = remove_other(text)
    text = rem2(text)
    text = lemming(text)
    #text = stemming(text)
    return(text)

In [249]:
#full clean
df_test['clean_text'] = df_test['text'].apply(lambda x: Full_clean(x)) #Clean the original data
df['clean_text'] = df['text'].apply(lambda x: Full_clean(x)) #Clean the original data

df_test

,index,text,clean_text
0,1,mmasepala fa maemo a a kgethegileng a letlelela kgato eo,mmasepala fa maemo a a kgethegileng a letlelela kgato eo
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye imitlolo engezelelako ukuqedelela ukutloliswa kwesibawo sakho,uzakwaziswa ngokufaneleko nakungafuneka eminye imitlolo engezelelako ukuqedelela ukutloliswa kwesibawo sakho
2,3,tshivhumbeo tshi fana na ngano dza vhathu,tshivhumbeo tshi fana na ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini naticocisana,kube inja nelikati betingevakala kutsi titsini naticocisana
4,5,winste op buitelandse valuta,winste op buitelandse valuta
...,...,...,...
5677,5678,you mark your ballot in private,you mark your ballot in private
5678,5679,ge o ka kgetha ka bowena go se šomiše mofani ka tirelo yo a kgethilwego wa gems dsp gona tefelo mmogo e tla šomišwa tefelo mmogo yeo e ka nna ya ba phapano magareng ga hlwahlwa ya tirelo le yeo ge...,ge o ka kgetha ka bowena go se šomiše mofani ka tirelo yo a kgethilwego wa gem dsp gona tefelo mmogo e tla šomišwa tefelo mmogo yeo e ka nna ya ba phapano magareng ga hlwahlwa ya tirelo le yeo gem...
5679,5680,e ka kopo etsa kgetho ya hao ka hloko hobane ha o na ho kgona ho fetola kgetho ena hara selemo ntle le tumello ya boto ya batshwaredi,e ka kopo etsa kgetho ya hao ka hloko hobane ha o na ho kgona ho fetola kgetho ena hara selemo ntle le tumello ya boto ya batshwaredi
5680,5681,tb ke bokudi ba pmb mme morero o tla lefella tlhahlobo ya ho sheba lefu lena leha ho le jwalo o tlameha ho fumana kalafo ya hao ya tb ditsing tsa mmuso,tb ke bokudi ba pmb mme morero o tla lefella tlhahlobo ya ho sheba lefu lena leha ho le jwalo o tlameha ho fumana kalafo ya hao ya tb ditsing tsa mmuso


## Model Building and Testing

In [250]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['lang_id'],test_size = 0.1,random_state=0)


In [251]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

MB = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop,analyzer='char',ngram_range=(5,5))),
               ('clf', MultinomialNB())])

MB = MB.fit(X_train, y_train)


MB_prediction = MB.predict(X_test) 
MB_accuracy = f1_score(y_test,MB_prediction, average='weighted')

print("\nEvaluation for MultinomialNB Data: ")
print("Training accuracy Score    : ",MB.score(X_train,y_train))
print("Testing accuracy Score : ",MB_accuracy )
print("\nClassification report: \n" ,classification_report(MB_prediction,y_test))


Evaluation for MultinomialNB Data: 
Training accuracy Score    :  0.99989898989899
Testing accuracy Score :  1.0

Classification report: 
               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       300
         eng       1.00      1.00      1.00       272
         nbl       1.00      1.00      1.00       312
         nso       1.00      1.00      1.00       277
         sot       1.00      1.00      1.00       299
         ssw       1.00      1.00      1.00       320
         tsn       1.00      1.00      1.00       295
         tso       1.00      1.00      1.00       299
         ven       1.00      1.00      1.00       306
         xho       1.00      1.00      1.00       308
         zul       1.00      1.00      1.00       312

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300



## Model Explanation

Naive Bayes classifiers are a collection of classification algorithms based on Bayes’ Theorem. It is not a single algorithm but a family of algorithms where all of them share a common principle, i.e. every pair of features being classified is independent of each other.

Basically, we are trying to find:

The probability of event A, given the event B is true. Event B is also termed as evidence.
P(A) is the priori of A (the prior probability, i.e. Probability of event before evidence is seen). The evidence is an attribute value of an unknown instance(here, it is event B).
P(A|B) is a posteriori probability of B, i.e. probability of event after evidence is seen.

# Create csv file

In [252]:
X_test_data = df_test['clean_text']#test dataset


test_pred = MB.predict(X_test_data)

df_test['lang_id'] = test_pred #store the predicted sentiment into the sentiment column

submission = df_test[['index','lang_id']] #define my dataframe called submission with two columns of tweetid and sentiment
submission.to_csv('submission.csv', index=False) # writing data to a CSV file
submission

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr
...,...,...
5677,5678,eng
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [253]:
submission.to_csv('submission.csv',index=False)